In [29]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce

from datetime import datetime
from hts import HTSRegressor
from hts.utilities.load_data import load_hierarchical_sine_data
import hts.functions
import collections
from hts.hierarchy import HierarchyTree

ModuleNotFoundError: No module named 'prophet'

## Hierarchical ARIMA by ags2 and ags5 

In [19]:
df = pd.read_csv('./../final_dfs/for_modeling/df_final_date_wide_with_cluster.csv', converters={'cluster': str, 'ags5': str}) 
df = df[['cluster','ags5','unemployment_rate', 'date']]
df

,cluster,ags5,unemployment_rate,date
0,0,01001,8.2,2019-01-01
1,0,01001,7.8,2019-02-01
2,0,01001,7.8,2019-03-01
3,0,01001,7.6,2019-04-01
4,0,01001,7.7,2019-05-01
...,...,...,...,...
11223,0,16077,6.7,2020-12-01
11224,0,16077,7.3,2021-01-01
11225,0,16077,7.3,2021-02-01
11226,0,16077,7.2,2021-03-01


create dictionary to specify hierarcy: 

In [20]:
d = df.groupby('cluster')['ags5'].apply(lambda x: list(set(x))).to_dict()
d

{'0': ['08421',
  '05316',
  '09572',
  '03151',
  '08315',
  '07133',
  '10046',
  '16062',
  '09372',
  '01051',
  '08136',
  '05558',
  '09775',
  '16065',
  '08117',
  '12053',
  '09678',
  '09274',
  '03359',
  '08327',
  '03401',
  '08425',
  '09671',
  '14626',
  '01061',
  '05974',
  '03252',
  '09373',
  '12062',
  '09673',
  '12072',
  '14522',
  '07318',
  '09277',
  '08417',
  '15081',
  '06440',
  '07140',
  '08128',
  '05966',
  '09174',
  '12063',
  '03356',
  '16072',
  '09161',
  '03402',
  '01057',
  '09679',
  '07132',
  '09571',
  '03353',
  '07339',
  '14524',
  '09173',
  '08211',
  '08436',
  '09574',
  '15088',
  '03256',
  '09273',
  '16052',
  '05958',
  '09463',
  '09363',
  '16067',
  '03159',
  '05166',
  '09772',
  '07334',
  '10043',
  '12061',
  '07332',
  '08326',
  '12060',
  '07111',
  '08237',
  '06632',
  '07335',
  '09675',
  '03251',
  '03103',
  '08231',
  '08435',
  '08337',
  '09779',
  '15001',
  '06635',
  '09263',
  '09778',
  '05366',
  '16

In [21]:
level_names = ['cluster', 'ags5']
hierarchy = [['cluster'], ['ags5']]
df_wide, sum_mat, sum_mat_labels = hts.functions.get_hierarchichal_df(df,
                                                                    level_names=level_names,
                                                                    hierarchy=hierarchy,
                                                                    date_colname='date',
                                                                    val_colname='unemployment_rate')
df_wide

cluster_ags5,0_01001,0_01004,0_01051,0_01053,0_01054,0_01055,0_01056,0_01057,0_01058,0_01059,...,16068,16069,16070,16071,16072,16073,16074,16075,16076,16077
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,8.2,8.3,6.4,4.9,6.3,6.3,4.8,4.7,4.3,4.7,...,6.2,3.9,5.5,4.5,4.2,5.9,4.8,5.5,5.3,8.0
2019-02-01,7.8,8.2,6.3,5.0,6.3,6.2,4.8,4.5,4.2,4.6,...,6.1,3.7,5.5,4.4,4.0,5.9,4.7,5.4,5.2,8.0
2019-03-01,7.8,8.0,6.0,4.8,5.5,5.6,4.6,4.3,4.0,4.4,...,5.7,3.4,5.3,4.2,3.7,5.7,4.4,5.2,4.9,7.6
2019-04-01,7.6,7.9,5.5,4.6,4.5,4.8,4.5,4.0,3.9,4.3,...,5.3,3.2,4.9,4.0,3.7,5.3,4.2,4.8,4.6,7.2
2019-05-01,7.7,8.0,5.2,4.6,4.2,4.5,4.5,3.9,4.0,4.2,...,5.4,3.2,4.7,3.9,3.7,5.3,4.1,4.6,4.5,7.1
2019-06-01,7.7,7.9,5.0,4.5,4.1,4.4,4.5,3.8,3.8,4.2,...,5.2,3.2,4.7,3.8,3.6,5.1,3.9,4.5,4.4,7.0
2019-07-01,8.1,8.0,5.2,4.8,4.3,4.6,4.7,4.0,4.1,4.4,...,5.2,3.3,4.8,3.9,3.7,5.2,3.9,4.5,4.5,7.0
2019-08-01,8.3,7.9,5.3,4.7,4.1,4.5,4.5,3.9,4.0,4.3,...,5.5,3.4,4.9,4.0,3.8,5.2,4.1,4.5,4.5,7.0
2019-09-01,8.3,7.6,5.1,4.6,4.0,4.3,4.3,3.8,3.8,4.1,...,5.3,3.2,4.7,3.8,3.8,4.9,3.9,4.1,4.4,6.5


add columns with the total and sum for each bundesland: 

In [22]:
df_wide['total'] = df_wide.sum(axis=1)
for key in d.keys():
    df_wide[key] = df_wide[[col for col in df_wide.columns if col.startswith(key)]].sum(axis=1)

In [23]:
df_wide.index = pd.to_datetime(df_wide.index)

add total to dict:

In [24]:
d['total'] = list(d.keys())

create tree to hierarcy:

In [93]:
ht = HierarchyTree.from_nodes(d, df_wide)
print(ht)

Exception ignored in: <function tqdm.__del__ at 0x7fc8591254c0>
Traceback (most recent call last):
  File "/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/tqdm/std.py", line 1144, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 

train the model: 

In [30]:
hp = HTSRegressor(model='prophet', revision_method='BU', n_jobs=0)
hp.fit(df_wide, d)

Fitting models:   0%|          | 0/405 [00:00<?, ?it/s]Importing plotly failed. Interactive plots will not work.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:   0%|          | 1/405 [00:01<13:11,  1.96s/it]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:   0%|          | 2/405 [00:02<06:36,  1.02it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to over

INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:   6%|▌         | 24/405 [00:12<03:21,  1.89it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:   6%|▌         | 25/405 [00:13<03:13,  1.96it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:   6%|▋         | 26/405 [00:13<03:05,  2.04it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with 

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  12%|█▏        | 48/405 [00:24<03:11,  1.87it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  12%|█▏        | 49/405 [00:24<03:03,  1.94it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  12%|█▏        | 50/405 [00:25<02:44,  2.16it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with w

Fitting models:  18%|█▊        | 71/405 [00:34<02:16,  2.45it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  18%|█▊        | 72/405 [00:35<02:14,  2.48it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  18%|█▊        | 73/405 [00:35<02:18,  2.40it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greate

INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  23%|██▎       | 95/405 [00:45<02:10,  2.38it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  24%|██▎       | 96/405 [00:45<01:58,  2.60it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  24%|██▍       | 97/405 [00:46<02:13,  2.30it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with 

...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  82%|████████▏ | 332/405 [03:07<00:33,  2.17it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  82%|████████▏ | 333/405 [03:07<00:33,  2.16it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  82%|████████▏ | 334/405 [03:08<00:36,  1.96it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet wit

Fitting models:  88%|████████▊ | 355/405 [03:56<00:26,  1.89it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  88%|████████▊ | 356/405 [03:56<00:25,  1.91it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  88%|████████▊ | 357/405 [03:57<00:25,  1.89it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints gre

INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  94%|█████████▎| 379/405 [04:07<00:14,  1.81it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  94%|█████████▍| 380/405 [04:07<00:13,  1.82it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models:  94%|█████████▍| 381/405 [04:08<00:13,  1.83it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet wi

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models: 100%|█████████▉| 403/405 [04:18<00:00,  2.16it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models: 100%|█████████▉| 404/405 [04:19<00:00,  2.21it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 21.
Fitting models: 100%|██████████| 405/405 [04:19<00:00,  1.56it/s]


HTSRegressor(n_jobs=0,
             revision_method=<hts.revision.RevisionMethod object at 0x7fca3a825f70>)

try another approach to train the model: 

In [66]:
#hp = HTSRegressor(model='sarimax', revision_method='BU', n_jobs=0, order=(2,2,2))
#hp.fit(df_wide, d)

Fitting models:   0%|          | 0/405 [00:00<?, ?it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Fitting models:   4%|▍         | 16/405 [00:01<00:47,  8.11it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Fitting models:   7%|▋         | 28/405 [00:03<00:51,  7.35it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax

Fitting models:  75%|███████▍  | 302/405 [00:35<00:11,  9.20it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Fitting models:  76%|███████▌  | 308/405 [00:36<00:12,  7.88it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Fitting models:  79%|███████▉  | 321/405 [00:37<00:09,  8.91it/s]/Users/amitsasson/venv/DSSGx/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Fitting models:  88%|████████▊ | 358/405 [00:42<00:05,  8.63it/s]/Users/amitsasson/venv/

HTSRegressor(model='sarimax', n_jobs=0,
             revision_method=<hts.revision.RevisionMethod object at 0x7fc82ce3f490>)

In [ ]:
hp.predict(steps_ahead=8)

Fitting models:  26%|██▌       | 106/405 [02:27<06:48,  1.36s/it]

Calaulate in sample MSE for the kreise

In [ ]:
resid = hp.hts_result.residuals
def entries_to_remove(entries, the_dict):
    for key in entries:
        if key in the_dict:
            del the_dict[key]
    return the_dict

resid = entries_to_remove(list(d.keys()), resid)
resid = list(resid.values()) 
resid = [item for sublist in resid for item in sublist]
(np.array(resid)**2).mean()

In [ ]:
clusters = df["cluster"].unique().tolist()
clusters

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(20, 10))

def plot_results(cols, axes, preds):
    axes = np.hstack(axes)
    for ax, col in zip(axes, cols):
        preds[col].plot(ax=ax, label="Predicted")
        df_wide[col].plot(ax=ax, label="Observed")

        ax.legend()
        ax.set_title(col)
        ax.set_xlabel("Date")
        ax.set_ylabel(" ")
    
plot_results(['05315','1','09561','15083','01061','03353'], axes, predicted)
plt.tight_layout()

## Try multivariate (dont read that, only a draft): 

In [59]:
exogenous = {k: ['unemployment_rate', 'number_of_companies_administration'] for k in df.columns if k not in ['unemployment_rate', 'number_of_companies_administration']}
ht = HierarchyTree.from_nodes(d, df, exogenous=exogenous)
print(ht)

KeyError: "None of [Index(['total'], dtype='object')] are in the [columns]"

In [ ]:
resid = hp.hts_result.residuals
def entries_to_remove(entries, the_dict):
    for key in entries:
        if key in the_dict:
            del the_dict[key]
    return the_dict

resid = entries_to_remove(list(d.keys()), resid)
resid = list(resid.values()) 
resid = [item for sublist in resid for item in sublist]
(np.array(resid)**2).mean()

In [ ]:
df = pd.read_csv('./../final_dfs/for_modeling/df_final_date_wide.csv', converters={'ags2': str, 'ags5': str}) 
df = df[['ags2','ags5','unemployment_rate', 'date', 'number_of_companies_administration']]

In [ ]:
level_names = ['ags2', 'ags5']
hierarchy = [['ags2'], ['ags5']]
df_wide, sum_mat, sum_mat_labels = hts.functions.get_hierarchichal_df(df,
                                                                    level_names=level_names,
                                                                    hierarchy=hierarchy,
                                                                    date_colname='date',
                                                                    val_colname=['unemployment_rate','number_of_companies_administration'])
df_wide